In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#import the modules 
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras import optimizers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import KFold,GridSearchCV

In [ ]:
train_data = pd.read_csv("../input/train.csv")
test_data = pd.read_csv("../input/test.csv")

In [ ]:
def visDataHist(data,feature_columns):
    
    plt.figure(figsize=(15,10))
    for i,key in enumerate(feature_columns):
        plt.subplot(len(feature_columns)/3+1,3,i+1)
        data[key].hist()
        plt.title(key)
    plt.show()

In [ ]:
visDataHist(train_data,[col for col in train_data.columns if train_data[col].dtype != 'object'])

In [ ]:
def check_value_counts(data):
    data_keys = data.keys()
    
    for key in data_keys:
        print(key)
        print(data[key].value_counts())
        print('----------------------------')

In [ ]:
def check_missing_values(data):
    print(data.isnull().sum())

In [ ]:
check_missing_values(train_data)
check_missing_values(test_data)

In [ ]:
check_value_counts(train_data[[col for col in train_data.columns if col != 'PassengerId']])
print('*'*100,'\n')
check_value_counts(test_data[[col for col in test_data.columns if col != 'PassengerId']])

In [ ]:
def preProcessingData(data,feature_columns):
    data = data[feature_columns]
    
    my_imputer = SimpleImputer()
    
    data['Age_was_missing'] = data['Age'].isnull()
    data['Age_was_missing'] = data['Age_was_missing'].apply(lambda v:1 if v==True else 0)
    data['Sex'] = data['Sex'].map(lambda v: 1 if v=='male' else 0)
    if data['Fare'].isnull().sum():
        data['Fare'] = pd.Series((my_imputer.fit_transform(np.array(data['Fare']).reshape(-1,1))).reshape(-1,),name='Fare')
    
    bins = np.linspace(0,100,5)
    data.Fare = pd.Series(np.digitize(data.Fare,bins),name='Fare')
    
    
    data = data.rename(columns = {'Name':'Name_title'})
    data['Name_title'] = pd.Series(data.Name_title.str.split(',').str[1].str.split('.').str[0].str.strip())
    data.Name_title.replace(to_replace = ['Dr','Major','Capt','Sir','Rev','Mlle','Col','Mme','Jonkheer','Don','the Countess','Dona'],value = 'Master', inplace = True)
    data.Name_title.replace(to_replace = ['Lady','Ms'], value = 'Miss', inplace = True)
    
    data['Cabin'].iloc[data.Cabin.loc[data.Cabin.notnull()].index] = 1
    data['Cabin'] = data.Cabin.fillna(0)
    #data['Cabin'] = data['Cabin'].apply(lambda v: v[0])
    
    data['FamSize'] = data.SibSp + data.Parch + 1
    data.FamSize.replace(to_replace = [1], value = 'single', inplace = True)
    data.FamSize.replace(to_replace = [2,3,4,5], value = 'family', inplace = True)
    data.FamSize.replace(to_replace = [6,7,8,11], value = 'large_family', inplace = True)
    
    data.Embarked = data.Embarked.fillna('C')
    data.Embarked.replace(to_replace = 'C', value = 1 , inplace = True)
    data.Embarked.replace(to_replace = 'S', value = 2 , inplace = True)
    data.Embarked.replace(to_replace = 'Q', value = 3 , inplace = True)
    data = data.drop(['SibSp'],axis=1)
    
    return data

In [ ]:
def correlationDraw(data,subFeature,mainFeature):
    
    correlation = data.loc[:,subFeature]
    correlation = correlation.agg(LabelEncoder().fit_transform)
    correlation[mainFeature] = data[mainFeature]

    correlationData = correlation.corr()
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    cax = ax.matshow(correlationData,vmin=-1,vmax=1)
    
    for(i,j),val in np.ndenumerate(correlationData):
        ax.text(j,i,'{:0.1f}'.format(val),ha='center',va='center')
    
    ticks = np.arange(0,len(subFeature)+1,1)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    
    ax.set_xticklabels(subFeature+mainFeature)
    ax.set_yticklabels(subFeature+mainFeature)

    plt.show()

In [ ]:
TargetLabel = train_data['Survived']
feature_columns = ['Pclass','Name','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked']

preprocessed_test_data = preProcessingData(test_data,feature_columns)
preprocessed_train_data = preProcessingData(train_data,feature_columns)

merged_data = pd.concat([preprocessed_train_data,preprocessed_test_data])

correlationDraw(merged_data,['Pclass','Name_title','Sex','FamSize','Fare','Cabin','Embarked'],['Age'])

In [ ]:
check_missing_values(merged_data)
merged_data.Age = merged_data.groupby(['Name_title','Pclass'])['Age'].transform(lambda v: v.fillna(v.median()))
Age_bins = np.linspace(20,60,3)
merged_data.Age = pd.Series(np.digitize(merged_data.Age,Age_bins),name='Age')


In [ ]:
check_value_counts(merged_data)

In [ ]:
def deep_model_build(input_shape,train_x,train_y,test_x,test_y):

    deep_model = Sequential()
    deep_model.add(Dense(32,input_shape=input_shape))
    deep_model.add(Activation('relu'))
    deep_model.add(Dropout(0.2))

    deep_model.add(Dense(64))
    deep_model.add(Activation('relu'))
    deep_model.add(Dropout(0.2))

    deep_model.add(Dense(32))
    deep_model.add(Activation('relu'))
    deep_model.add(Dropout(0.2))
    
    deep_model.add(Dense(20))
    deep_model.add(Activation('relu'))
    
    
    deep_model.add(Dense(2))
    deep_model.add(Activation('softmax'))

    my_optimizer = optimizers.Adadelta()
    deep_model.compile(loss='mean_squared_error',optimizer=my_optimizer,metrics=['accuracy'])
    deep_model.optimizer.lr=0.1
    train_y_temp = np_utils.to_categorical(train_y,2)
    test_y_temp = np_utils.to_categorical(test_y,2)
    
    history = deep_model.fit(train_x,train_y_temp,batch_size=24,epochs=150,verbose=0,validation_data=(test_x,test_y_temp))
    print('Train accuracy:',np.max(history.history['acc']))
    print('Validation accuracy:',np.max(history.history['val_acc']))
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training data')
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.legend(['training','valiation'],loc='lower right')
    plt.show()
    return deep_model

In [ ]:
Numerical_merged_data = pd.get_dummies(merged_data)
processed_train_data = Numerical_merged_data.iloc[:preprocessed_train_data.shape[0],:]
processed_test_data = Numerical_merged_data.iloc[preprocessed_train_data.shape[0]:,:]
#train_x = processed_train_data.iloc[:int(len(processed_train_data)*0.8),:]
#test_x = processed_train_data.iloc[int(len(processed_train_data)*0.8):,:]
#train_y = TargetLabel[0:int(len(TargetLabel)*0.8)]
#test_y = TargetLabel[int(len(TargetLabel)*0.8):]
train_x,test_x,train_y,test_y = train_test_split(processed_train_data,TargetLabel,test_size=0.2,random_state=1)

In [ ]:
ntrain = len(processed_train_data)
ntest = len(processed_test_data)
#ntrain = len(train_x)
#ntest = len(test_x)
kf = KFold(n_splits=5,random_state=2017)
def get_oof(classifier,train_x,train_y,test_x):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.zeros((5,ntest))
    
    for i,(train_idx,test_idx) in enumerate(kf.split(train_x)):
        kf_train_x = train_x.iloc[train_idx]
        kf_train_y = train_y[train_idx]
        kf_test_x = train_x.iloc[test_idx]
        
        classifier.fit(kf_train_x,kf_train_y)
        oof_train[test_idx] = classifier.predict(kf_test_x)
        oof_test_skf[i,:] = classifier.predict(test_x)
    
    oof_test = oof_test_skf.mean(axis=0)
    
    return oof_train.reshape(-1,1),oof_test.reshape(-1,1)


In [ ]:
def tuneParams(classifier,params,train_x,train_y):
    md = GridSearchCV(classifier,params,cv=10,scoring='accuracy',n_jobs=-1)
    md.fit(train_x,train_y)
    best_params,best_score = md.best_params_,np.round(md.best_score_*100,2)
    
    return best_params,best_score

In [ ]:
xgbParams = {'max_depth':[3,5,8,10],'learning_rate':[0.01,0.1,0.5],'n_estimator':[100,300,500],'gamma':[0.01,0.05,0.1,0.5]}
ABParams = {'n_estimators':[100,300,500],'learning_rate':[0.01,0.5,0.1,0.5]}
RFParams = {'n_estimators':[10,30,50],'max_depth':[3,5,8,10],'min_samples_split':[2,5,10],'min_samples_leaf':[2,4,10],'random_state':[5]}
ETParams = {'n_estimators':[100,300,500],'max_depth':[3,8,12],'min_samples_leaf':[2,4,9],'verbose':[0]}
KNNParams = {'n_neighbors':[3,8,14],'leaf_size':[2,5,9],'weights':['uniform']}
SVCParams = {'C':[0.01,0.1,0.5],'gamma':[0.01,0.2]}

In [ ]:
xgb_best_Params,xgb_best_score = tuneParams(XGBClassifier(),xgbParams,processed_train_data,TargetLabel)
AB_best_Params,AB_best_score = tuneParams(AdaBoostClassifier(),ABParams,processed_train_data,TargetLabel)
RF_best_Params,RF_best_score = tuneParams(RandomForestClassifier(),RFParams,processed_train_data,TargetLabel)
ET_best_Params,ET_best_score = tuneParams(ExtraTreesClassifier(),ETParams,processed_train_data,TargetLabel)
KNN_best_Params,KNN_best_score = tuneParams(KNeighborsClassifier(),KNNParams,processed_train_data,TargetLabel)
SVC_best_Params,SVC_best_score = tuneParams(SVC(),SVCParams,processed_train_data,TargetLabel)
print("XGB:",xgb_best_Params,xgb_best_score)
print("AdaBoost:",AB_best_Params,AB_best_score)
print("RandomForest:",RF_best_Params,RF_best_score)
print("ExtraTrees:",ET_best_Params,ET_best_score)
print("KNeighbors:",KNN_best_Params,KNN_best_score)
print("SVC:",SVC_best_Params,SVC_best_score)

In [ ]:
xgb_oof_train,xgb_oof_test = get_oof(XGBClassifier(**xgb_best_Params),processed_train_data,TargetLabel,processed_test_data)
AB_oof_train,AB_oof_test = get_oof(AdaBoostClassifier(**AB_best_Params),processed_train_data,TargetLabel,processed_test_data)
RF_oof_train,RF_oof_test = get_oof(RandomForestClassifier(**RF_best_Params),processed_train_data,TargetLabel,processed_test_data)
ET_oof_train,ET_oof_test = get_oof(ExtraTreesClassifier(**ET_best_Params),processed_train_data,TargetLabel,processed_test_data)
KNN_oof_train,KNN_oof_test =  get_oof(KNeighborsClassifier(**KNN_best_Params),processed_train_data,TargetLabel,processed_test_data)
SVC_oof_train,SVC_oof_test = get_oof(SVC(**SVC_best_Params),processed_train_data,TargetLabel,processed_test_data)

In [ ]:
final_train = np.concatenate((xgb_oof_train,AB_oof_train,RF_oof_train,ET_oof_train,KNN_oof_train,SVC_oof_train),axis=1)
final_test = np.concatenate((xgb_oof_test,AB_oof_test,RF_oof_test,ET_oof_test,KNN_oof_test,SVC_oof_test),axis=1)

In [ ]:
LRParams = {'penalty':['l1','l2'],'C':[0.01,0.05,0.1,0.2]}
LR_best_Params,LR_best_score = tuneParams(LogisticRegression(),LRParams,final_train,TargetLabel)
print("LR:",LR_best_Params,LR_best_score)

In [ ]:
LR_model = LogisticRegression(**LR_best_Params)
LR_model.fit(final_train,TargetLabel)
prediction = LR_model.predict(final_test)

In [ ]:
deep_output = pd.DataFrame({'PassengerId':test_data['PassengerId'],
                            'Survived':prediction})
deep_output.to_csv('submission_v12.csv',index=False)